# 📱 Kenya Mobile Transactions Deep Dive
## Analyzing the Engine of Kenya's Digital Economy

**Advanced analytics on mobile money and digital payment trends.**

This notebook explores the vast landscape of mobile transactions in Kenya, a global leader in mobile money. It includes:
- 📊 **Transaction Volume & Value Analysis**: Understanding the scale of the digital economy.
- 📈 **Growth & Trend Analysis**: Tracking the adoption and evolution of mobile payments.
- 🤖 **Predictive Modeling**: Forecasting future transaction volumes.
- 💡 **Economic Impact Assessment**: How mobile money impacts GDP and financial inclusion.
- 💳 **Comparative Analysis**: Mobile payments vs. traditional banking (Cheques, EFTs).

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_theme(style="darkgrid", palette="plasma")
plt.rcParams['figure.figsize'] = (14, 7)

print("📦 Libraries imported successfully!")
print("🚀 Starting Mobile Transactions Deep Dive...")

In [ ]:
# Load and Prepare Mobile Transactions Data
def load_and_clean_mobile_data():
    """Load and prepare all relevant mobile transaction datasets"""
    
    try:
        # Load Mobile Payments data
        mobile_payments_df = pd.read_csv('../data/raw/Mobile Payments.csv', skiprows=2)
        print(f"✅ Loaded Mobile Payments data: {mobile_payments_df.shape}")
        
        # Load Cheques and EFTs for comparison
        cheques_efts_df = pd.read_csv('../data/raw/Cheques  EFTs.csv', skiprows=2)
        print(f"✅ Loaded Cheques & EFTs data: {cheques_efts_df.shape}")
        
        # --- Data Cleaning ---
        
        # Clean Mobile Payments
        mobile_payments_df.columns = ['Date', 'Agents_KES_M', 'Agents_Count', 'Person_to_Person_KES_M', 'Person_to_Person_Count']
        mobile_payments_df['Date'] = pd.to_datetime(mobile_payments_df['Date'], errors='coerce')
        for col in mobile_payments_df.columns[1:]:
            mobile_payments_df[col] = pd.to_numeric(mobile_payments_df[col].astype(str).str.replace(',', ''), errors='coerce')
        mobile_payments_df.set_index('Date', inplace=True)
        
        # Add total columns
        mobile_payments_df['Total_Value_KES_M'] = mobile_payments_df['Agents_KES_M'] + mobile_payments_df['Person_to_Person_KES_M']
        mobile_payments_df['Total_Count'] = mobile_payments_df['Agents_Count'] + mobile_payments_df['Person_to_Person_Count']
        
        # Clean Cheques & EFTs
        cheques_efts_df.columns = ['Date', 'Cheques_Value_KES_M', 'Cheques_Count', 'EFT_Value_KES_M', 'EFT_Count']
        cheques_efts_df['Date'] = pd.to_datetime(cheques_efts_df['Date'], errors='coerce')
        for col in cheques_efts_df.columns[1:]:
            cheques_efts_df[col] = pd.to_numeric(cheques_efts_df[col].astype(str).str.replace(',', ''), errors='coerce')
        cheques_efts_df.set_index('Date', inplace=True)
        
        print("🧼 Data cleaning and preparation complete.")
        
        return mobile_payments_df.dropna(), cheques_efts_df.dropna()
        
    except FileNotFoundError as e:
        print(f"⚠️  Error: {e}. One of the data files was not found. Please check the path.")
        return None, None

# Load the datasets
mobile_data, traditional_data = load_and_clean_mobile_data()

if mobile_data is not None:
    print("\n📋 Mobile Payments Data Overview:")
    print(mobile_data.head())

In [ ]:
# Exploratory Data Analysis (EDA) and Visualization
def create_mobile_dashboard(mobile_df, traditional_df):
    """Create a comprehensive dashboard for mobile transactions"""
    
    if mobile_df is None or traditional_df is None:
        print("Cannot create dashboard due to missing data.")
        return
        
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "📈 Total Value of Mobile Transactions (KES Million)",
            "📊 Transaction Count (Mobile vs. Traditional)",
            "💹 Growth of Mobile Payments vs. EFTs",
            " seasonal Decomposition of Mobile Transaction Value"
        ],
        specs=[[{}, {}], [{"secondary_y": True}, {"secondary_y": True}]]
    )
    
    # 1. Total Value of Mobile Transactions
    fig.add_trace(
        go.Scatter(x=mobile_df.index, y=mobile_df['Total_Value_KES_M'], name='Total Mobile Value', mode='lines', line=dict(color='rebeccapurple')),
        row=1, col=1
    )
    
    # 2. Transaction Count Comparison
    mobile_annual_count = mobile_df['Total_Count'].resample('Y').sum()
    eft_annual_count = traditional_df['EFT_Count'].resample('Y').sum()
    cheque_annual_count = traditional_df['Cheques_Count'].resample('Y').sum()
    
    fig.add_trace(go.Bar(x=mobile_annual_count.index, y=mobile_annual_count, name='Mobile Transactions'), row=1, col=2)
    fig.add_trace(go.Bar(x=eft_annual_count.index, y=eft_annual_count, name='EFT Transactions'), row=1, col=2)
    fig.add_trace(go.Bar(x=cheque_annual_count.index, y=cheque_annual_count, name='Cheque Transactions'), row=1, col=2)
    fig.update_layout(barmode='group')
    
    # 3. Growth of Mobile vs EFTs
    mobile_growth = mobile_df['Total_Value_KES_M'].pct_change(12) * 100
    eft_growth = traditional_df['EFT_Value_KES_M'].pct_change(12) * 100
    
    fig.add_trace(go.Scatter(x=mobile_growth.index, y=mobile_growth, name='Mobile Value Growth (%)', mode='lines'), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=eft_growth.index, y=eft_growth, name='EFT Value Growth (%)', mode='lines'), row=2, col=1, secondary_y=True)
    
    # 4. Seasonal Decomposition
    decomposition = seasonal_decompose(mobile_df['Total_Value_KES_M'].dropna(), model='additive', period=12)
    fig.add_trace(go.Scatter(x=decomposition.trend.index, y=decomposition.trend, name='Trend', mode='lines'), row=2, col=2, secondary_y=False)
    fig.add_trace(go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, name='Seasonality', mode='lines'), row=2, col=2, secondary_y=True)
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="📱 Kenya Mobile Transactions Dashboard",
        title_x=0.5,
        showlegend=True
    )
    fig.update_yaxes(title_text="KES Million", row=1, col=1)
    fig.update_yaxes(title_text="Number of Transactions", row=1, col=2)
    fig.update_yaxes(title_text="YoY Growth (%)", row=2, col=1)
    fig.update_yaxes(title_text="Trend", row=2, col=2, secondary_y=False)
    fig.update_yaxes(title_text="Seasonal Component", row=2, col=2, secondary_y=True)
    
    fig.show()
    print("✅ Mobile transactions dashboard created successfully!")

# Generate and display the dashboard
create_mobile_dashboard(mobile_data, traditional_data)

In [ ]:
# Predictive Modeling for Mobile Transaction Volume
def transaction_forecasting_model(df):
    """Forecast future mobile transaction volume"""
    
    print("\n🤖 PREDICTIVE MODELING FOR TRANSACTION VOLUME")
    print("="*40)
    
    # Prepare data
    df_monthly = df['Total_Count'].resample('M').sum().reset_index()
    df_monthly['Time'] = np.arange(len(df_monthly))
    
    # Features and target
    X = df_monthly[['Time']]
    y = df_monthly['Total_Count']
    
    # Train model
    model = LinearRegression()
    model.fit(X, y)
    
    # Forecast
    future_periods = 24 # 2 years
    last_time = X['Time'].iloc[-1]
    future_df = pd.DataFrame({'Time': np.arange(last_time + 1, last_time + 1 + future_periods)})
    forecast = model.predict(future_df)
    
    # Create future dates for plotting
    future_dates = pd.date_range(start=df_monthly['Date'].iloc[-1], periods=future_periods + 1, freq='M')[1:]
    
    # Visualization
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_monthly['Date'], y=y, name='Historical Transaction Count', mode='lines', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=future_dates, y=forecast, name='Forecasted Transaction Count', mode='lines', line=dict(color='red', dash='dash')))
    
    fig.update_layout(
        title="Mobile Transaction Count Forecast (Next 2 Years)",
        xaxis_title="Date",
        yaxis_title="Number of Transactions",
        height=500
    )
    fig.show()
    
    print("✅ Transaction volume forecasting completed.")
    return model, forecast

# Run the forecasting model
volume_model, volume_forecast = transaction_forecasting_model(mobile_data)

# Economic Impact and Conclusion

**Key Insights:**
- Mobile transactions have seen exponential growth, far outpacing traditional methods like cheques and even EFTs.
- The value and volume of mobile payments are a significant component of Kenya's economy.
- There is a clear seasonal pattern, likely tied to holidays and economic cycles.
- The growth trajectory indicates that mobile payments will continue to be the dominant form of transaction for the foreseeable future.

**Economic Impact:**
- **Financial Inclusion:** Mobile money has provided access to financial services for millions of unbanked Kenyans.
- **Velocity of Money:** The ease and speed of transactions increase the velocity of money, stimulating economic activity.
- **Formalization of Economy:** Small-scale businesses can participate more easily in the formal economy.
- **Data for Policymaking:** The vast amount of transaction data provides valuable insights for economic planning.

**Conclusion:**
Kenya's mobile payment ecosystem is a cornerstone of its modern economy. Its continued growth is vital for financial inclusion, economic efficiency, and innovation. Policymakers and businesses must continue to support and leverage this powerful platform.